# Pruebas con las bases de datos



## Preamble 

In [1]:
import $file.sparksession
import sparksession._
import spark.implicits._
import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._, func._

Loading spark-stubs
Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/07/03 10:03:43 INFO SparkContext: Running Spark version 2.4.3
20/07/03 10:03:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
20/07/03 10:03:44 INFO SparkContext: Submitted application: 7dbef658-fc68-4c09-8660-9772eb49b3d5
20/07/03 10:03:44 INFO SecurityManager: Changing view acls to: jovyan
20/07/03 10:03:44 INFO SecurityManager: Changing modify acls to: jovyan
20/07/03 10:03:44 INFO SecurityManager: Changing view acls groups to: 
20/07/03 10:03:44 INFO SecurityManager: Changing modify acls groups to: 
20/07/03 10:03:44 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(jovyan); groups with view permissions: Set(); users  with modify permissions: Set(jovyan); groups with modify permissions: Set()
20/07/03 10:03:45 INFO Utils: Successfully started service 'sparkD

20/07/03 10:03:45 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0-sources.jar at spark://b0149c19624d:39919/jars/scala-collection-compat_2.12-2.0.0-sources.jar with timestamp 1593770625879
20/07/03 10:03:45 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.0.0/scala-collection-compat_2.12-2.0.0.jar at spark://b0149c19624d:39919/jars/scala-collection-compat_2.12-2.0.0.jar with timestamp 1593770625889
20/07/03 10:03:45 INFO SparkContext: Added JAR file:/home/jovyan/.cache/coursier/v1/https/repo1.maven.org/maven2/org/scala-lang/modules/scala-xml_2.12/1.2.0/scala-xml_2.12-1.2.0-sources.jar at spark://b0149c19624d:39919/jars/scala-xml_2.12-1.2.0-sources.jar with timestamp 1593770625889
20/07/03 10:03:45 INFO SparkContext: Added JAR file:/home/jovyan/.cache

20/07/03 10:03:46 INFO BlockManagerMaster: Registered BlockManager BlockManagerId(driver, b0149c19624d, 36525, None)
20/07/03 10:03:46 INFO BlockManager: Initialized BlockManager: BlockManagerId(driver, b0149c19624d, 36525, None)


Spark UI

import $file.$           

import sparksession._

import spark.implicits._

import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.types._, func._

In [ ]:
//import $ivy.`org.vegas-viz::vegas:0.3.9`

In [ ]:
//val sqlContext = SparkSession.builder().getOrCreate();

## DataFrame

In [2]:
def readCsv(input: String ): DataFrame ={
    spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("data/" + input + ".csv")
}

defined function readCsv

In [7]:
val temperatureByCountry : DataFrame = readCsv("GlobalLandTemperaturesByCountry")
val disasterNumber = readCsv("number-of-natural-disaster-events").drop($"Code")
val disasterEconomic: DataFrame = readCsv("economic-damage-from-natural-disasters").drop($"Code")
val deathByCountry= readCsv("deaths-natural-disasters-ihme").withColumn("Deaths", $"Deaths".cast("Long"))

load at cmd1.sc:5

1 / 1

load at cmd1.sc:5

2 / 2

load at cmd1.sc:5

...

load at cmd1.sc:5

1 / 1

load at cmd1.sc:5

2 / 2

temperatureByCountry: DataFrame = [dt: timestamp, AverageTemperature: double ... 2 more fields]
disasterNumber: DataFrame = [Entity: string, Year: int ... 1 more field]
disasterEconomic: DataFrame = [Entity: string, Year: int ... 1 more field]
deathByCountry: DataFrame = [Entity: string, Code: string ... 2 more fields]

In [8]:
val disasterDeath : DataFrame = readCsv("number-of-deaths-from-natural-disasters").drop($"Code")
val deathPercentByCountry: DataFrame = readCsv("share-deaths-from-natural-disasters")
                                .withColumn("Deaths(Percent) (%)",format_number($"Deaths(Percent) (%)", 2).cast("Double"))


load at cmd1.sc:5

1 / 1

load at cmd1.sc:5

2 / 2

load at cmd1.sc:5

1 / 1

load at cmd1.sc:5

2 / 2

disasterDeath: DataFrame = [Entity: string, Year: int ... 1 more field]
deathPercentByCountry: DataFrame = [Entity: string, Code: string ... 2 more fields]

In [9]:
val earthquakes : DataFrame = readCsv("significant-earthquakes").drop($"Code")
val volcano : DataFrame = readCsv("significant-volcanic-eruptions").drop($"Code")

load at cmd1.sc:5

1 / 1

load at cmd1.sc:5

2 / 2

load at cmd1.sc:5

1 / 1

load at cmd1.sc:5

2 / 2

earthquakes: DataFrame = [Entity: string, Year: int ... 1 more field]
volcano: DataFrame = [Entity: string, Year: int ... 1 more field]

## Algunas queries 

In [12]:
disasterNumber.select($"Entity", $"Year", $"Number").show

show at cmd11.sc:1

1 / 1

+--------------------+----+------+
|              Entity|Year|Number|
+--------------------+----+------+
|All natural disas...|1900|     5|
|All natural disas...|1901|     2|
|All natural disas...|1902|     9|
|All natural disas...|1903|     8|
|All natural disas...|1904|     2|
|All natural disas...|1905|     4|
|All natural disas...|1906|    17|
|All natural disas...|1907|     5|
|All natural disas...|1908|     4|
|All natural disas...|1909|    11|
|All natural disas...|1910|    15|
|All natural disas...|1911|     5|
|All natural disas...|1912|     7|
|All natural disas...|1913|     5|
|All natural disas...|1914|     6|
|All natural disas...|1915|     7|
|All natural disas...|1916|     2|
|All natural disas...|1917|     6|
|All natural disas...|1918|     7|
|All natural disas...|1919|     4|
+--------------------+----+------+
only showing top 20 rows



In [ ]:
disasterNumber.filter($"Entity" =!="All natural disasters")
         .groupBy($"Year")
         .sum("Number")
         .orderBy($"Year")
         .show

disasterNumber.filter($"Entity" ==="All natural disasters")
         .show


disasterNumber.filter($"Entity" =!="All natural disasters")
         .groupBy($"Year", $"Entity")
         .sum("Number")
         .orderBy($"Year".desc)
         .show

In [ ]:
temperatureByCountry.filter($"AverageTemperature" > -1000000)
            .groupBy($"dt")
            .avg("AverageTemperature")
            .orderBy($"dt")
            .limit(20)
            .show

In [11]:
//para seccion 3.2
disasterNumber.filter($"Entity" ==="All natural disasters")
val temperatureByCountryYear = temperatureByCountry
                                .withColumn("dt", date_format(col("dt"), "yyyy")
                                .cast("Int"))
val temperatureAVGTotal = temperatureByCountryYear
                    .groupBy($"dt")
                    .avg("AverageTemperature")
                    .orderBy($"dt")
temperatureAVGTotal.map(_.getAs[Double](1))

res10_0: Dataset[Row] = [Entity: string, Year: int ... 1 more field]
temperatureByCountryYear: DataFrame = [dt: int, AverageTemperature: double ... 2 more fields]
temperatureAVGTotal: Dataset[Row] = [dt: int, avg(AverageTemperature): double]
res10_3: Dataset[Double] = [value: double]

In [16]:
disasterNumber
    .filter($"Entity" ==="All natural disasters")
    .orderBy($"Year")
    .select($"Year",$"Number")
    .map(_.getInt(0))

res15: Dataset[Int] = [value: int]

## Queries con explicaciones

In [ ]:
//Relacionar numero de desastres con sus daños economicos

val disasterNumberDamage =disasterNumber.join(disasterEconomic, Seq("Entity","Year"))
disasterNumberDamage.filter($"Entity" =!="All natural disasters").limit(10).show

In [ ]:
//Creo que no funciona porque en ambos casos, en el campo Code es null

disasterNumber.join(disasterEconomic, Seq("Entity","Code","Year")).limit(10).show

In [ ]:
// Modificar la columna dt de tipo timestamp (yyyy-MM-dd hh-mm-ss) a solo de años 

val temperatureByCountryYear:DataFrame = temperatureByCountry.withColumn("dt", date_format(col("dt"), "yyyy"))

In [ ]:
// tabla con temperetura media del año por pais 
//ordenado primero por el año y luego por pais
val temperatureAVG = temperatureByCountryYear
                    .groupBy($"dt",$"Country")
                    .avg("AverageTemperature")
                    .orderBy($"dt", $"Country")
temperatureAVG.limit(10).show

In [14]:
//Relacionar el numero total de muertos con numero de desastres

val disasterNumberDeath =disasterNumber.join(disasterDeath, Seq("Entity","Year"))
disasterNumberDeath.limit(10).show


disasterNumberDeath: DataFrame = [Entity: string, Year: int ... 2 more fields]

In [ ]:
//Relacionar numero de muertos por pais con su porcentage

val deathNumberPercent =deathByCountry.join(deathPercentByCountry, Seq("Entity","Year"))
deathNumberPercent.filter($"Year">2000).filter($"Entity" === "Haiti").limit(10).show

## Solución con Plotly

In [3]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.2`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._


import $ivy.$                                      


import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._


In [ ]:

//purebas

// val consulta= disasterNumber.filter($"Entity" =!="All natural disasters")
//          .groupBy($"Year",$"Entity")
//          .sum("Number")
//          .orderBy($"Year")
// val x=consulta.select($"Year").collectAsList.toArray.mkString("!!!").replace("[","").replace("]","").split("!!!").toSeq
// val y = consulta.select($"sum(Number)").collectAsList.toArray.mkString("!!!").replace("[","").replace("]","").split("!!!").toSeq
// Bar(x,y).plot()

// val trace1 = Scatter(x,y)


// trace1.plot()

In [4]:
val disasterNumber = readCsv("number-of-natural-disaster-events").drop($"Code")

load at cmd1.sc:5

1 / 1

load at cmd1.sc:5

2 / 2

disasterNumber: DataFrame = [Entity: string, Year: int ... 1 more field]

In [5]:
// 1.¿Cómo evoluciona el número de desastres naturales según avanza los años?
val aux = disasterNumber.filter($"Entity" ==="All natural disasters").orderBy($"Year").select($"Year",$"Number")
Bar(aux.map(_.getInt(0)).collect.toSeq,
    aux.map(_.getInt(1)).collect.toSeq).plot()

collect at cmd4.sc:2

1 / 1

collect at cmd4.sc:2

1 / 1

collect at cmd4.sc:2

...

collect at cmd4.sc:2

1 / 1

collect at cmd4.sc:2

64 / 64

aux: DataFrame = [Year: int, Number: int]
res4_1: String = "plot-7226470b-a4ab-443c-92ff-83c20b38fe1f"

In [6]:
//Para seccion 3.3
val aux1 = aux.collect
Bar(aux1.map(_.getInt(0)).toSeq,
    aux1.map(_.getInt(1)).toSeq).plot()

collect at cmd5.sc:1

1 / 1

collect at cmd5.sc:1

1 / 1

collect at cmd5.sc:1

64 / 64

aux1: Array[Row] = Array(
  [1900,5],
  [1901,2],
  [1902,9],
  [1903,8],
  [1904,2],
  [1905,4],
  [1906,17],
  [1907,5],
  [1908,4],
  [1909,11],
  [1910,15],
  [1911,5],
  [1912,7],
  [1913,5],
  [1914,6],
  [1915,7],
  [1916,2],
  [1917,6],
  [1918,7],
  [1919,4],
  [1920,7],
  [1921,2],
  [1922,6],
  [1923,12],
  [1924,4],
  [1925,6],
  [1926,15],
  [1927,12],
  [1928,17],
  [1929,6],
  [1930,14],
  [1931,12],
  [1932,8],
  [1933,10],
  [1934,9],
  [1935,11],
  [1936,11],
  [1937,7],
...
res5_1: String = "plot-86349315-24c7-4eac-99df-00edeed415f9"

In [ ]:
// 2.¿Cómo evoluciona el daño económico causado por los desastres naturales?
val aux = disasterEconomic.filter($"Entity" ==="All natural disasters").orderBy($"Year").select($"Year",$"Money")
Bar(aux.map(_.getInt(0)).collect.toSeq,aux.map(_.getLong(1)).collect.toSeq).plot()

In [ ]:
//3.¿Existe alguna relación entre el número de desastres naturales con el daño económico?

val diasasterNumberEconomic=disasterNumber.join(disasterEconomic, Seq("Entity","Year"))
val trace1= Scatter(
        diasasterNumberEconomic.map(_.getAs[Int]("Number")).collect.toSeq, 
        diasasterNumberEconomic.map(_.getAs[Long]("Money")).collect.toSeq,
        mode = ScatterMode(ScatterMode.Markers)
        )
        .plot()

In [ ]:
// Modificar la columna dt de tipo timestamp (yyyy-MM-dd hh-mm-ss) a solo de años 

val temperatureByCountryYear:DataFrame = temperatureByCountry.withColumn("dt", date_format(col("dt"), "yyyy").cast("Int"))

In [ ]:
// 4.¿Cómo evoluciona la temperatura media anual global?
val temperatureAVGTotal = temperatureByCountryYear
                    .groupBy($"dt")
                    .avg("AverageTemperature")
                    .orderBy($"dt")
Scatter(
    temperatureAVGTotal.map(_.getInt(0)).collect.toSeq,
    temperatureAVGTotal.map(_.getAs[Double](1)).collect.toSeq
).plot()

In [ ]:
// 5.¿Cómo evoluciona la temperatura media anual de cada país?
val temperatureAVG = temperatureByCountryYear
                    .groupBy($"dt",$"Country")
                    .avg("AverageTemperature")
                    .orderBy($"dt", $"Country")
                    


In [ ]:
//Cojo dos paises como ejemplo
val albania =temperatureAVG.filter($"Country"==="Albania")
val trace1 = Bar(
  albania.map(_.getAs[Int]("dt")).collect.toSeq,
  albania.map(_.getAs[Double]("avg(AverageTemperature)")).collect.toSeq,
  name = "Albania"
)

val andorra =temperatureAVG.filter($"Country"==="Andorra")
val trace2 = Bar(
  andorra.map(_.getAs[Int]("dt")).collect.toSeq,
  andorra.map(_.getAs[Double]("avg(AverageTemperature)")).collect.toSeq,
  name = "Andorra"
)


val data = Seq(trace1, trace2)

data.plot()

// val layout = Layout( 
//   barmode = BarMode.Group
// )

// Plotly.plot("div-id", data, layout)

In [ ]:
//6.¿Existe alguna relación entre la temperatura media anual global 
//con el número de desastres naturales?
val aux1 = disasterNumber.filter($"Entity" ==="All natural disasters")

val aux = aux1.join(temperatureAVGTotal, aux1("Year")===temperatureAVG("dt"))

Scatter(
    aux.map(_.getAs[Double](4)).collect.toSeq,
    aux.map(_.getAs[Int](2)).collect.toSeq,
    mode = ScatterMode(ScatterMode.Markers)
).plot()

In [ ]:
//7.¿Cómo evoluciona el número de muertes anual 
//causados por los desastres naturales?
val aux = deathByCountry
       .groupBy($"Year")
       .sum("Deaths")
       .orderBy($"Year")
Bar(
    aux.map(_.getAs[String](0)).collect.toSeq,
    aux.map(_.getAs[Long](1)).collect.toSeq
).plot()

In [ ]:
//8.¿Cómo evoluciona la media del porcentaje de muertes?
val aux = deathPercentByCountry
       .groupBy($"Year")
       .avg("Deaths(Percent) (%)")
       .orderBy($"Year")
Bar(
    aux.map(_.getAs[String](0)).collect.toSeq,
    aux.map(_.getAs[Double](1)).collect.toSeq
).plot()

In [ ]:
//9.¿Cuál es el país con más muertes por los desastres naturales?

  deathByCountry
  .groupBy($"Entity")
  .sum("Deaths")
  .orderBy($"sum(Deaths)".desc)
  .show

In [ ]:
//10.¿Existe alguna relación entre el número de desastres naturales con el número 
//total de muertos anuales causados por los desastres naturales?

val disasterNumberDeath =disasterNumber.join(disasterDeath, Seq("Entity","Year")).filter($"Entity"==="All natural disasters")


Scatter(
        disasterNumberDeath.map(_.getAs[Int]("Number")).collect.toSeq, 
        disasterNumberDeath.map(_.getAs[Int]("Deaths")).collect.toSeq,
        mode = ScatterMode(ScatterMode.Markers)
        )
        .plot()

In [ ]:
// 11.¿Cómo evolucionan cada tipo de desastres naturales según avanza el tiempo?

 val aux = disasterNumber.filter($"Entity" =!="All natural disasters")
         .groupBy($"Year", $"Entity")
         .sum("Number")
         .orderBy($"Year")

//        val data= aux.groupByKey(_.getAs[String]("Entity"))
//          .mapGroups((key,value) => Scatter(value.map(_.getAs[Int]("Year")).toSeq,
//                                           value.map(_.getAs[Int]("Number")).toSeq,
//                                           name = key))

val trace1 = Scatter(
  aux.filter($"Entity" === "Flood").map(_.getInt(0)).collect.toSeq,
  aux.filter($"Entity" === "Flood").map(_.getLong(2)).collect.toSeq,
  name="Flood"
)


val trace2 = Scatter(
  aux.filter($"Entity" === "Extreme weather").map(_.getInt(0)).collect.toSeq,
  aux.filter($"Entity" === "Extreme weather").map(_.getLong(2)).collect.toSeq,
  name="Extreme weather"
)

val data = Seq(trace1, trace2)

data.plot()



In [ ]:
//12.¿Qué tipo de desastres naturales provoca mayores muertos?
disasterNumber.join(disasterDeath, Seq("Entity","Year"))
       .groupBy($"Entity")
       .sum("Deaths")
       .filter($"Entity" =!= "All natural disasters")
       .orderBy($"sum(Deaths)".desc).show

In [ ]:
//13.¿Qué tipo de desastres naturales provoca mayores daños económicos?
disasterEconomic
       .groupBy($"Entity")
       .sum("Money")
       .filter($"Entity" =!= "All natural disasters")
       .orderBy($"sum(Money)".desc)
       .show

In [ ]:
//14.¿Cuándo hubo más concentración de terremotos significativos?
Bar(
    earthquakes.filter($"Year">1700).map(_.getAs[Int](1)).collect.toSeq,
    earthquakes.filter($"Year">1700).map(_.getAs[Int](2)).collect.toSeq
).plot()

In [ ]:
//15.¿Cuándo hubo más concentración de erupciones volcánicas?
Bar(
    volcano.filter($"Year">1700).map(_.getAs[Int](1)).collect.toSeq,
    volcano.filter($"Year">1700).map(_.getAs[Int](2)).collect.toSeq
).plot()

## Para capitulo 4.2 Rendimiento

In [ ]:
//Para seccion 4.2.1
//dataset de prueba
val lista = List((1,3,"hola"),(2,4,"tarde"),
        (3,2,"noche"),(4,5,"buenas"),(5,10,"mundo"))
val listDS = lista.toDS

In [ ]:
//quedar con los elementos que tenga el 
// primer componente mayor que 2
lista.filter(_._1 > 2)
//los que tenga el elemento en la primera columna 
//mayor que 2
listDS.filter($"_1" > 2)
// quedar con el segundo y tercer componente
lista.map{ case(n,f,s) => (f,s)}
//Quedar con la segunda y tercera columna
listDS.select($"_2",$"_3").collect

In [ ]:
//Para seccion 4.2.2
val temperatureByCity = readCsv("GlobalLandTemperaturesByCity")

In [ ]:
//Guardar en formato parquet 
temperatureByCity.write.mode("overwrite").parquet("data/temperatureByCity.parquet")

In [ ]:
val ds = readCsv("GlobalLandTemperaturesByCity").select($"City")

In [ ]:
ds.explain

In [ ]:
val df: DataFrame = 
    spark.read.parquet("data/temperatureByCity.parquet").select($"City")

In [ ]:
df.explain

In [ ]:
//guardamos en memoria con partition
temperatureByCity
    .write.mode("overwrite")
    .partitionBy("Country")
    .parquet("data/country.parquet")

In [ ]:
//guardamos en memoria con repartition previo
temperatureByCity
    .repartition($"Country")
    .write.mode("overwrite")
    .partitionBy("Country")
    .parquet("data/countryre.parquet")

In [ ]:
//filter sin partition
val df: DataFrame = 
    spark.read.parquet("data/temperatureByCity.parquet")
            .filter($"Country" === "Denmark" && $"AverageTemperature" >10)

In [ ]:
df.show

In [ ]:
df.explain

In [ ]:
//filter con partition
val ds = spark.read.parquet("data/country.parquet")
            .filter($"Country" === "Denmark" && $"AverageTemperature" >10)

In [ ]:
ds.show

In [ ]:
ds.explain

## Pruebas con Evilplot
no funciona

In [ ]:
interp.repositories() ++= Seq(
  coursierapi.MavenRepository.of("https://dl.bintray.com/cibotech/public")
)


In [ ]:
import $ivy.`com.cibo::evilplot-repl:0.7.0`
def showPlot(plot: com.cibo.evilplot.geometry.Drawable) =
  Image.fromRenderedImage(plot.asBufferedImage, Image.PNG)

In [ ]:
// 1.	¿Cómo evoluciona el número de desastres naturales según avanza los años?
val aux = disasterNumber.filter($"Entity" ==="All natural disasters").orderBy($"Year").select($"Year",$"Number")


In [ ]:
import com.cibo.evilplot.colors.RGB
import com.cibo.evilplot.geometry.{Align, Drawable, Extent, Rect, Text}
import com.cibo.evilplot.plot._
import com.cibo.evilplot.plot.aesthetics.DefaultTheme.{DefaultFonts, DefaultTheme}
import com.cibo.evilplot.plot.renderers.BarRenderer



BarChart
  .custom(aux.map(_.getInt(1)).collect.toSeq.map(Bar.apply)
  )
   .title("Ejemplo")
   .xAxis(aux.map(_.getInt(0)))
   .yAxis()
   .frame()
   .bottomLegend()
   .render()